In [1]:
import hashlib
import time

# Building the first block class

In [2]:
class Block:

    def __init__(self, index, proof_no, prev_hash, data, timestamp=None):
        self.index = index
        self.proof_no = proof_no
        self.prev_hash = prev_hash
        self.data = data
        self.timestamp = timestamp or time.time()

    @property
    def calculate_hash(self):
        block_of_string = "{}{}{}{}{}".format(self.index, self.proof_no,
                                              self.prev_hash, self.data,
                                              self.timestamp)

        return hashlib.sha256(block_of_string.encode()).hexdigest()

    def __repr__(self):
        return "{} - {} - {} - {} - {}".format(self.index, self.proof_no,
                                               self.prev_hash, self.data,
                                               self.timestamp)
    


# Sample Block Information

In [3]:
{
    "index": 2,
    "proof": 21,
    "prev_hash": "6e27587e8a27d6fe376d4fd9b4edc96c8890346579e5cbf558252b24a8257823",
    "transactions": [
        {'sender': '0', 'recipient': 'Alison', 'quantity': 1}
    ],
    "timestamp": 1521646442.4096143
}

{'index': 2,
 'proof': 21,
 'prev_hash': '6e27587e8a27d6fe376d4fd9b4edc96c8890346579e5cbf558252b24a8257823',
 'transactions': [{'sender': '0', 'recipient': 'Alison', 'quantity': 1}],
 'timestamp': 1521646442.4096143}

# Block Chain Class

In [4]:
class BlockChain:                                       #blockchain instantiated 

    def __init__(self):           
        self.chain = []
        self.current_data = []
        self.nodes = set()
        self.construct_genesis()
        
    def construct_genesis(self):                        #Initial Block                 
        self.construct_block(proof_no=0, prev_hash=0)

    def construct_block(self, proof_no, prev_hash):     
        block = Block(
        index=len(self.chain),
        proof_no=proof_no,
        prev_hash=prev_hash,
        data=self.current_data)
        self.current_data = []

        self.chain.append(block)
        return block
    
    @staticmethod
    def check_validity(block, prev_block):            #Checking Validity and anomalies 
        if prev_block.index + 1 != block.index:
            return False

        elif prev_block.calculate_hash != block.prev_hash:
            return False

        elif not BlockChain.verifying_proof(block.proof_no, prev_block.proof_no):
            return False

        elif block.timestamp <= prev_block.timestamp:
            return False
    
        return True
    
    def new_data(self, sender, recipient, quantity):    #New Transaction 
        self.current_data.append({
            'sender': sender,
            'recipient': recipient,
            'quantity': quantity
        })
        return True
    
    @staticmethod
    def proof_of_work(last_proof):                     #Proof of work concept to prevent blockchain abuse 
        '''this simple algorithm identifies a number f' such that hash(ff') contain 4 leading zeroes
         f is the previous f'
         f' is the new proof
        '''
        proof_no = 0
        while BlockChain.verifying_proof(proof_no, last_proof) is False:
            proof_no += 1

        return proof_no


    @staticmethod
    def verifying_proof(last_proof, proof):
        #verifying the proof: does hash(last_proof, proof) contain 4 leading zeroes?
        guess = f'{last_proof}{proof}'.encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == "0000"
    @property
    def latest_block(self):                       #Latest Block 
        return self.chain[-1]

# Testing of  code and adding a new transaction 

In [5]:
blockchain = BlockChain()

#Mining

print("***Mining RAA_Coin about to start***")
print(blockchain.chain)

last_block = blockchain.latest_block
last_proof_no = last_block.proof_no
proof_no = blockchain.proof_of_work(last_proof_no)

#Adding new transaction

blockchain.new_data(
    sender="0",  #it implies that this node has created a new block
    recipient="Alison",  #let's send Alison some coins!
    quantity=
    1,  #creating a new block (or identifying the proof number) is awarded with 1
)

last_hash = last_block.calculate_hash
block = blockchain.construct_block(proof_no, last_hash)

#Mining Sucessful

print("***Mining RAA_Coin has been successful***")
print(blockchain.chain)


***Mining RAA_Coin about to start***
[0 - 0 - 0 - [] - 1657533504.671346]
***Mining RAA_Coin has been successful***
[0 - 0 - 0 - [] - 1657533504.671346, 1 - 88914 - c262d2b8ff306d58b12c8ab7b1e4900e9799f5666cd6e0b8231ce1fb1e5a3612 - [{'sender': '0', 'recipient': 'Alison', 'quantity': 1}] - 1657533504.9468632]
